In [1]:
import os
import pandas as pd
import re
from collections import ChainMap
import json

In [2]:
with open('../neuroscience/Papers_neuroscience_all.json', "r") as read_file:
    papers = json.load(read_file)

In [3]:
listPapers0=[p['id'] for p in papers if p['queryLevel']==0]
listPapers1=[p['id'] for p in papers if p['queryLevel']==1]

In [7]:
listPapers=listPapers0+listPapers1

In [6]:
your_path = '../../../../../../Volumes/TOSHIBA EXT/Simplified_files'
print( os.listdir(your_path))
infile=your_path+'/paperAuthors_simplified.txt'

['.DS_Store', 'affiliations.json', 'Authors_simplified.txt', 'journals.csv', 'paperAbstracts_simplified.txt', 'paperAuthors_simplified.txt', 'paperReferences_simplified.txt', 'Papers_simplified.txt']


In [8]:
selDf=pd.DataFrame()
chunksize = 10 ** 6
for chunk in pd.read_csv(infile, chunksize=chunksize, names=['paperId', 'authorId'],sep=';'):
    selDf=selDf.append(chunk[chunk['paperId'].isin(listPapers)])

In [9]:
d2=selDf.groupby('paperId')['authorId'].apply(list)

In [10]:
e=0
for p in papers:
    try:
        p['authors']=d2[int(p['id'])]
    except:
        e=e+1

In [11]:
with open('../neuroscience/Papers_neuroscience_all.json', 'w') as fout:
    json.dump(papers , fout)

# preparation author affiliation csv

In [9]:
papers[0]

{'id': 2038431747,
 ' created': ' "2016-06-24"',
 ' doi': '10.1162/NECO.1994.6.3.441',
 ' volume': ' "6"',
 ' appearsInJournal': ' 207023548',
 ' citationCount': ' "16"',
 ' issueIdentifier': ' "3"',
 ' title': ' "Topology learning solved by extended objects: A neural network model"',
 ' referenceCount': ' "7"',
 ' endingPage': ' "458"',
 ' startingPage': ' "441"',
 ' publicationDate': ' "1994-05-01"',
 ' publisher': ' "MIT Press"',
 'queryLevel': 0,
 'authors': [2163422316, 2419355596, 2289589767]}

In [5]:
ll=[el['authors'] for el in papers if 'authors' in el.keys()]
flat_list = [item for sublist in ll for item in sublist]
allAu=list(set(flat_list))

In [10]:
p0=[p for p in papers if p['queryLevel']==0]
p1=[p for p in papers if p['queryLevel']==1]

ll0=[el['authors'] for el in p0 if 'authors' in el.keys()]
ll1=[el['authors'] for el in p1 if 'authors' in el.keys()]

flat_list = [item for sublist in ll0 for item in sublist]
au0=set(flat_list)

flat_list = [item for sublist in ll1 for item in sublist]
au1=set(flat_list)

In [13]:
only1=list(au1-au0)

queryLevel={}
for el in list(au0):
    queryLevel[el]=0
for el in only1:
    queryLevel[el]=1

In [14]:
your_path = '../../../../../../Volumes/TOSHIBA EXT/Simplified_files'
infile=your_path+'/Authors_simplified.txt'

In [15]:
auDf=pd.DataFrame()
chunksize = 10 ** 6
for chunk in pd.read_csv(infile, chunksize=chunksize, names=['authorId', 'feature','value'],sep=';'):
    auDf=auDf.append(chunk[chunk['authorId'].isin(allAu)])

In [16]:
aff={}
name={}
affDf=auDf[auDf['feature']=='affiliation']
nameDf=auDf[auDf['feature']=='name']

for (u,v) in zip(affDf['authorId'],affDf['value']):
    aff[u]=v
    
for (u,v) in zip(nameDf['authorId'],nameDf['value']):
    name[u]=v

In [17]:
auAffDf=pd.DataFrame()
auAffDf['authorId']=name.keys()
auAffDf['name']=[name[i] for i in name.keys()]
auAffDf['affiliation']=[aff[i] if i in aff.keys() else 'NONE' for i in name.keys()]
auAffDf['queryLevel']=[queryLevel[i] for i in name.keys()]

In [18]:
auAffDf.to_csv('../neuroscience/authorId_name_affiliation_neuroscience.csv', index=False)